# Movies Clustering

### Description
Notebook to generate a Clustering Model for tests (Unsupervising)

In [1]:
import numpy as np
import pandas as pd
# import seaborn as sns

In [2]:
import sys
print(sys.version)

3.8.10 (default, Mar 15 2022, 12:22:08) 
[GCC 9.4.0]


In [ ]:
import os
ARTIFACTS_PATH = '../../artifacts/bigml/'
os.makedirs(ARTIFACTS_PATH, exist_ok=True) # Create path if not exists

### Save our BigML Username and Api Key to our environment to access the API

In [ ]:
import os
os.environ['BIGML_USERNAME'] = "elizaglez"
os.environ['BIGML_API_KEY'] = "2064c7cbf0a35661eb33602d6ecd264aecdfd520"
os.environ['BIGML_CRED'] = "username=$BIGML_USERNAME;api_key=$BIGML_API_KEY"
os.environ['BIGML_PROJECT'] = "project/59ef803eaf447f0b1d001156"

### Creat our main API object that all the main functions will utilize.

In [ ]:
from bigml.api import BigML
api = BigML(project="project/59ef803eaf447f0b1d001156")

In [ ]:
data_source = api.create_source("../../data/movies/movies_filtered_cluster.csv")
api.ok(data_source)

In [ ]:
data = api.create_dataset(data_source)
api.ok(data)

## Test-Train Split

In [ ]:
train_dataset = api.create_dataset(data,
                                   {"name": "cpu",
                                    "sample_rate": 0.8, "seed": "my seed",
                                    "tags": ["cpu_anomaly"]})
api.ok(train_dataset)

## Creating Models

### Kmeans Clustering

In [ ]:
model_name = "clusters_bigml"
desc = "Movies Clustering"
model_out_file_name = model_name + ".json"
model_out_path = os.path.join(ARTIFACTS_PATH, model_out_file_name)

In [ ]:
cluster = api.create_cluster(data, {"name": "Movie Clusters",
                                    "k": 100,
                                    "tags":[model_name],
                                    "field_scales": {"000002": 1, "000004":3},
                                    "excluded_fields":["000000"],
                                    "default_numeric_value":"zero"})
api.ok(cluster)

In [ ]:
# assigning an anomaly score to it
from bigml.cluster import Cluster
cluster_id = cluster["object"]["resource"]
local_cluster = Cluster(cluster_id)


In [ ]:
api.export_last(model_name,
                resource_type="cluster",
                filename=model_out_path)


### Testing

#### Known cluster

In [ ]:
input_data = {"original_title": "Toy Story", "genres": "Animation Comedy Family", "adult": False,
              "overview": "Led by Woody, Andy's toys live happily in his room until Andy's birthday brings Buzz Lightyear onto the scene. Afraid of losing his place in Andy's heart, Woody plots against Buzz. But when circumstances separate Buzz and Woody from their owner, the duo eventually learns to put aside their differences."}

data = local_cluster.centroid(input_data)
print(data['centroid_name'])

#### Unkown movie

In [ ]:
input_data = {"original_title": "The Spanish Warrior", "genres": "Action", "adult": False,
              "overview": "A movie about a warrior that saves spain from disaster"}

data = local_cluster.centroid(input_data)
print(data['centroid_name'])